In [1]:
# Import Statements

%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchsample
from torchsample import transforms as ts_transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
from tensorboardX import SummaryWriter
from datetime import datetime
import importlib

#from torchsample.transforms import RangeNorm

import functions.fine_tune as ft

plt.ion()   # interactive mode

/home/mtezcan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1 - Load Data

Before running the code, the data should be downloaded and foldered in the way that is usable for imagefolder function of the PyTorch. The following code assumes that the main directory for the dataset is 'data_dir' and it includes subdirectories for all of the separate classes.

For details on how to create those folders, pleaserefer to 'dataset/Folder_images.ipynb'

In [3]:
'''
Macros
'''
#uniform_sampler=False
batch_size=32
nclasses=10 #Number of output classes
split=200
random_seed=1
shuffle=True
dataset = 'abid'
CV = 0
train_val = None# (100000, 10000)

# Data augmentation and normalization for training 
# Just normalization for validation
'''data_transforms = {
    'train': transforms.Compose([
        #transforms.Scale(64),
        transforms.RandomCrop(56),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        #transforms.Scale(64),
        transforms.CenterCrop(56),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}'''

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


'''Create ImageFolder objects for training and test'''

if dataset == 'Hoarding':
    data_dir = '../datasets/hoarding/images'
elif dataset == 'Hoarding_frames':
    data_dir = '../datasets/hoarding/frames/all_inclusive'
else:
    data_dir = '../datasets/abid/train/balanced'  
    

if not CV == 0:  
    dset_train = datasets.ImageFolder(data_dir+'/train_val', data_transforms['train'])
    dset_val = datasets.ImageFolder(data_dir+'/train_val', data_transforms['val'])
    
    num_train = len(dset_train)
    indices = list(range(num_train))
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    splits = (num_train*np.linspace(0,1,CV+1)).astype(int)
    
    val_idx = [indices[splits[k]:splits[k+1]] for k in range(CV)]
    train_idx=[np.setdiff1d(indices,val_idx[k]) for k in range(CV)]
    '''Sampler functions for validation and training'''
    sampler_train = [torch.utils.data.sampler.SubsetRandomSampler(train_idx[k]) for k in range(CV)]
    sampler_val = [torch.utils.data.sampler.SubsetRandomSampler(val_idx[k]) for k in range(CV)]

    '''Define dataset loaders'''
    dset_loaders_arr = [{'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train[k],
                                                        num_workers=12),
                    'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val[k],
                                                        num_workers=12)} for k in range(CV)]
    dset_sizes={'train':int(len(dset_train)*(1-1/CV)),'val':int(len(dset_train)*(1/CV))}
    
    print(dset_sizes)
    print('OR')
    print('Number of training images '+str(len(val_idx)))
    print('Number of validation images '+str(len(train_idx)))

elif train_val:
    
    dset_train = datasets.ImageFolder(data_dir+'/train/0to5', data_transforms['train'])
    dset_val = datasets.ImageFolder(data_dir+'/train/0to5', data_transforms['val'])
    
    num_train = len(dset_train)
    indices = list(range(num_train))
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    train_idx = indices[:train_val[0]]
    val_idx = indices[-train_val[1]:]
    '''Sampler functions for validation and training'''
    sampler_train = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
    sampler_val = torch.utils.data.sampler.SubsetRandomSampler(val_idx)

    '''Define dataset loaders'''
    dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train,
                                                        num_workers=12),
                    'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val,
                                                        num_workers=12)}
    dset_sizes={'train':int(len(train_idx)),'val':int(len(val_idx))}
    
    print(dset_sizes)
    print('OR')
    print('Number of training images '+str(len(val_idx)))
    print('Number of validation images '+str(len(train_idx)))
else:
    dset_train = datasets.ImageFolder(data_dir+'/train/0to5', data_transforms['train'])
    dset_val = datasets.ImageFolder(data_dir+'/val/0to5', data_transforms['val'])

    '''Define dataset loaders'''
    dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,shuffle=True,
                                                        num_workers=8),
                    'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,
                                                        num_workers=8)}

    
    dset_sizes={'train':len(dset_train),'val':len(dset_val)}
    print(dset_sizes)

dset_classes = dset_train.classes
dset_classes_val = dset_val.classes
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")



print(dset_classes)
print(dset_classes_val)

if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')
    
'''kkk=1
for data in dset_loaders['train']:
    print(kkk)
    kkk+=1'''

{'train': 70000, 'val': 15000}
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
GPU is available


"kkk=1\nfor data in dset_loaders['train']:\n    print(kkk)\n    kkk+=1"

In [ ]:
dset_train.ConcatDataset(dset_val)
print(len(dset_train))

In [29]:
'''
Macros
'''
#uniform_sampler=False
batch_size=64
split=1000
random_seed=1
shuffle=True

# Data augmentation and normalization for training 
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


'''Create ImageFolder objects for training and test'''
data_dir = '../images'
#data_dir = '/home/mtezcan/Documents/amazon/bin_images/0to5'
dset_train = datasets.ImageFolder(data_dir+'/train', data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir+'/val', data_transforms['val'])
'''dset_train = datasets.ImageFolder(data_dir+'/0to5', data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir+'/0to5', data_transforms['val'])'''


'''Validation split'''
num_train = len(dset_train)
indices = list(range(num_train))
val_idx=np.loadtxt('./dataset/validation.txt').astype(np.int)
train_idx=np.setdiff1d(indices,val_idx)

'''if shuffle == True:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]'''

'''Sampler functions for validation and training'''
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(val_idx)

'''Define dataset loaders'''
dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,shuffle=True,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,shuffle=False,
                                                    num_workers=12)}
'''dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val,
                                                    num_workers=12)}'''

dset_sizes={'train':len(dset_train),'val':len(dset_val)}
#dset_sizes={'train':len(dset_train)-1000,'val':1000}
dset_classes = dset_train.classes
dset_classes_val = dset_val.classes
use_gpu = torch.cuda.is_available()

print(dset_sizes)
print('OR')
print('Number of training images '+str(len(val_idx)))
print('Number of validation images '+str(len(train_idx)))

print(dset_classes)
print(dset_classes_val)

if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


FileNotFoundError: [Errno 2] No such file or directory: '../images/train'

In [ ]:
print(np.max(val_idx))

Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:
inputs, classes = next(iter(dset_loaders['train']))
out = torchvision.utils.make_grid(inputs,nrow=8)
print('Size of the input tensors in one batch after grid is  '+str(out.size()))
plt.figure(figsize=(12,12))
ft.imshow(out, title=[dset_classes[x] for x in classes])

## 2- Log Keeping

This section includes the functions defined for the log keeping. Since CNNs require lots of trials, I found it easy to record the properties of the each trial with their performances in an excel file. I also added tnesorboard summaries for every trial and individual text files for showing the details of the network.

Below code declares the required parameters for the network which will be also used inside log keeping

In [ ]:
def make_coeff(n, metric, lmbda = 1):
    if metric is 'ccr':
        return [1]
    elif metric is 'ccr1':
        return [1, 1, 1]
    elif metric is 'mae':
        coeff = np.arange(1,n)/(n-1)
    elif metric is 'mse':
        coeff = np.zeros(n-1)
        coeff[0] = 2*n-3
        for k in range(1, n-1):
            coeff[k] = coeff[k-1] + 2*n - (2*(k+1)+1)
        coeff = coeff /((n-1)**2)
    else:
        print('Undefined Metric: ' + metric)
    coeff = np.concatenate((coeff, coeff[::-1][1:]), axis=0)
    coeff = coeff * lmbda
    coeff[n-2] = 1
    return coeff

In [12]:
def make_coeff(n, metric, loss='softmax', lmbda = 1):
    if metric is 'ccr':
        return np.eye(n)
    elif metric is 'ccr1':
        coeff = np.zeros((n,n))
        for k in range(n):
            coeff[k,np.maximum(k-1,0):np.minimum(k+2,n)]=1
        if loss is 'softmax':
            coeff_sum = np.sum(coeff,axis=1).reshape(-1,1)*np.ones((1,n))
            coeff = coeff/coeff_sum
    elif metric is 'mae':
        coeff = np.zeros((n,n))
        for k in range(n):
            alpha = 1/(n-1)
            row = np.zeros((1,n))
            for l in range(n-1):
                row_ = np.zeros((1,n))
                row_[0, np.maximum(k-l,0):np.minimum(k+l+1,n)] = 1
                if loss is 'softmax':
                    row_ = row_/np.sum(row_)
                row += row_*alpha
            coeff[k,:]=row
    elif metric is 'mse':
        coeff = np.zeros((n,n))
        for k in range(n):
            row = np.zeros((1,n))
            for l in range(n-1):
                row_ = np.zeros((1,n))
                row_[0, np.maximum(k-l,0):np.minimum(k+l+1,n)] = 1
                row += (2*l+1)*row_
                
            if loss is 'softmax':
                row = row / np.sum(row)
            elif loss is 'sigmoid':
                row = row / ((n**2)-(2*n)+1)
            else:
                 print('Undefined Loss: ' + str(loss))
            coeff[k,:]=row
    elif metric is 'test':
        coeff = np.ones((n,n))
        alpha = 1./45.
        for k in range(n):
            for l in range(n):
                coeff[k,l] = 1 - np.abs(k-l) * alpha
    else:
        print('Undefined Metric: ' + metric)
        
    coeff = np.eye(n)*(1-lmbda) + lmbda*coeff
        
    #coeff = np.concatenate((coeff, coeff[::-1][1:]), axis=0)
    #coeff = coeff * lmbda
    #coeff[n-2] = 1
    return coeff

coeff = make_coeff(10,'test',loss = 'softmax', lmbda=1)
print(coeff)

[[1.         0.97777778 0.95555556 0.93333333 0.91111111 0.88888889
  0.86666667 0.84444444 0.82222222 0.8       ]
 [0.97777778 1.         0.97777778 0.95555556 0.93333333 0.91111111
  0.88888889 0.86666667 0.84444444 0.82222222]
 [0.95555556 0.97777778 1.         0.97777778 0.95555556 0.93333333
  0.91111111 0.88888889 0.86666667 0.84444444]
 [0.93333333 0.95555556 0.97777778 1.         0.97777778 0.95555556
  0.93333333 0.91111111 0.88888889 0.86666667]
 [0.91111111 0.93333333 0.95555556 0.97777778 1.         0.97777778
  0.95555556 0.93333333 0.91111111 0.88888889]
 [0.88888889 0.91111111 0.93333333 0.95555556 0.97777778 1.
  0.97777778 0.95555556 0.93333333 0.91111111]
 [0.86666667 0.88888889 0.91111111 0.93333333 0.95555556 0.97777778
  1.         0.97777778 0.95555556 0.93333333]
 [0.84444444 0.86666667 0.88888889 0.91111111 0.93333333 0.95555556
  0.97777778 1.         0.97777778 0.95555556]
 [0.82222222 0.84444444 0.86666667 0.88888889 0.91111111 0.93333333
  0.95555556 0.97777

In [14]:
network='resnet18' #Initial network archtiecture.'loaded'forusing a saved network
networkName='resnet18_real_sgd_multisoft_August29  19:06:27' #Directory for the saved network
optimizer='sgd' #Optimizer function
iter_loc=19 #Number of the first column in the excel file for writing the results.
end_to_end=True #Booolean to decide whether to train the network end-to-end or not.
lr=0.01 #Initial learning rate
momentum=0.9
weight_decay=0.0005
lr_scheduler=ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=10 #Number of epoch for learning rate decay
pretrained=False 
mse_loss=False #Scalar MSE loss
#nclasses=9 #Number of output classes
metric = 'ccr'
coeff_lmbda =  0
multi_coeff = make_coeff(nclasses, metric, loss = 'sigmoid', lmbda = coeff_lmbda)
KL = False #KL divergence for porbability measure
cheng_lambda = 0


'''Multipliers for loss functions'''
single_loss=1.
multi_loss=0.

comment=' ' #Additional comments if any

algo = None

Let us define the functions for creating a text file and adding networkproperties to an excel file. 

In [15]:
def writeLog(logname):
    '''
    Creates a text file named Network_properties.txt inside runs/'logname'
    '''
    f=open('../results/tensorboard_runs/'+logname+'/Network_properties.txt','w')
    f.write('Batch size: '+str(batch_size)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Random seed: '+str(random_seed)+'\n')
    f.write('Shuffle: '+str(shuffle)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Network: '+network+'\n')
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    f.write('Criterion: '+crt+'\n')
    f.write('Learning rate: '+str(lr)+'\n')
    f.write('Momentum: '+str(momentum)+'\n')
    f.write('Leraning Rate Scheduler: '+str(lr_scheduler)+'\n')
    f.write('Leraning Rate Decay Period: '+str(lr_decay_epoch)+'\n')
    f.write('Network is pretrained: '+str(pretrained)+'\n')
    f.write('Network laoded from: '+networkName+'\n')
    f.write('MSE loss function: '+str(mse_loss)+'\n')
    f.close()

In [16]:
import openpyxl
import time

def writeLog_xlsx(logname='../results/logs.xlsx',iter_loc=19):
    '''
    Adds a line to logs.xlsx with the network properties and outcomes.
    :param iter_loc: First column to record the outcomes.
    '''
    book = openpyxl.load_workbook(logname)
    sheet = book.active
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    multi_coeff = make_coeff(nclasses, metric, loss = 'sigmoid', lmbda = coeff_lmbda)
    if network=='loaded':
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'), dataset, networkName,str(split), str(algo), str(metric),
               str(coeff_lmbda), str(random_seed),str(shuffle),
               optimizer, crt, str(multi_coeff), str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),
               str(pretrained), str(batch_size))
    else:
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'), dataset, network,str(split),str(algo), str(metric),
               str(coeff_lmbda), str(random_seed),str(shuffle),
               optimizer, crt, str(multi_coeff),str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),
               str(pretrained),str(batch_size))
    sheet.append(specs)
    current_row = sheet.max_row
    sheet.cell(row=current_row, column=iter_loc+5).value = comment
    book.save(logname)

## 3 - Training and Validation

In this part we will define the functions for training a CNN with different properties and loss functions.

The following function takes bunch of properties defined in the beginning of Section-2 as input and creates network using those properties.

In [ ]:
model_ft = models.resnet18(pretrained=pretrained)
#model_ft.layer1 = nn.Sequential()#model_ft.layer4
'''num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(3, 128, kernel_size=7, stride=2, padding=3,
                               bias=False)

model_ft.bn1 = nn.BatchNorm2d(128)
model_ft.avgpool = nn.AvgPool2d(2)
print(model_ft)
model_ft.fc = nn.Linear(num_ftrs, nclasses)'''


model_ft

In [17]:
def network_loader(comment=comment,
                    network=network,
                    networkName=networkName,
                    optimizer=optimizer,
                    iter_loc=iter_loc,
                    end_to_end=end_to_end,
                    lr=lr,
                    momentum=momentum,
                    weight_decay=weight_decay,
                    lr_scheduler=lr_scheduler,
                    lr_decay_epoch=lr_decay_epoch,
                    pretrained=pretrained,
                    mse_loss=mse_loss,
                    nclasses=nclasses):
    
    '''Load the network from pytorch'''
    if(network=='resnet18'):
        model_ft = models.resnet18(pretrained=pretrained)
        '''' model_ft.conv1 = nn.Conv2d(3, 128, kernel_size=7, stride=2, padding=3,
                               bias=False)

        model_ft.bn1 = nn.BatchNorm2d(128)
        model_ft.layer1 = nn.Sequential()
        model_ft.layer2 = nn.Sequential()
        model_ft.avgpool = nn.AvgPool2d(4)'''
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features #Change the last layer to adapt new classes
        if(mse_loss): #For MSE loss last layer should be a scalar
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet34'):
        model_ft = models.resnet34(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet50'):
        model_ft = models.resnet50(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet101'):
        model_ft = models.resnet101(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='alexnet'):
        model_ft = models.alexnet(pretrained=pretrained)
        num_ftrs = model_ft.classifier[6].out_features
        setattr(model_ft.classifier, '7', nn.ReLU(inplace=True))
        setattr(model_ft.classifier, '8', nn.Dropout())
        setattr(model_ft.classifier, '9', nn.Linear(num_ftrs,nclasses))

    elif(network=='loaded'):#Use the saved network when network='loaded'
        model_ft = torch.load('./saved_models/'+networkName)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, nclasses)
    else:
        raise ValueError('Undefined network '+network)

    if use_gpu:
        model_ft = model_ft.cuda()

    '''Define the optimizer function'''
    if(optimizer=='adam'):
        optimizer_ft = optim.Adam(model_ft.parameters(),lr=lr,weight_decay=weight_decay)
    elif(optimizer=='sgd'):
        if(end_to_end):
            optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum)
        else:
            optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=lr, momentum=momentum,weight_decay=weight_decay)
    return model_ft, optimizer_ft

Now define a simple function to be able to run our training in a single line

In [18]:
def write_epochs(result_log, logname):
    print(len(result_log))

    wb_tr = openpyxl.Workbook()
    ws_tr = wb_tr.active
    wb_val = openpyxl.Workbook()
    ws_val = wb_val.active
    print(logname)

    label_arr_tr = np.zeros((1,1))
    probs_arr_tr = np.zeros((1, nclasses))
    label_arr_val = np.zeros((1,1))
    probs_arr_val = np.zeros((1, nclasses))

    prev_epoch = 0
    for result in result_log:
        epoch = result[1]
        if not epoch == prev_epoch:
            ws_tr.append(['Epoch ' + str(prev_epoch)])
            ws_tr.append(label_arr_tr[1:].reshape(-1).tolist())
            ws_tr.append(np.argmax(probs_arr_tr[1:,:], axis=1).reshape(-1).tolist())
            for probs in probs_arr_tr[1:,:].T.tolist():
                ws_tr.append(probs)
            wb_tr.save('./runs/'+logname + '/train.xlsx')
            ws_val.append(['Epoch ' + str(prev_epoch)])
            ws_val.append(label_arr_val[1:].reshape(-1).tolist())
            ws_val.append(np.argmax(probs_arr_val[1:,:], axis=1).reshape(-1).tolist())
            for probs in probs_arr_val[1:,:].T.tolist():
                ws_val.append(probs)
            wb_val.save('./runs/'+logname + '/val.xlsx')

            del label_arr_tr, label_arr_val, probs_arr_tr, probs_arr_val
            label_arr_tr = np.zeros((1,1))
            probs_arr_tr = np.zeros((1, nclasses))
            label_arr_val = np.zeros((1,1))
            probs_arr_val = np.zeros((1, nclasses))
            prev_epoch = epoch

        label = np.asarray(result[2]).reshape(-1,1)
        scores = np.asarray(result[3])
        exp_scores = np.exp(scores - np.max(scores,axis=1).reshape(-1, 1)*np.ones(nclasses))
        probs = np.round(exp_scores/(np.sum(exp_scores,axis=1).reshape(-1, 1)*np.ones(nclasses)), decimals=2)
        if result[0] == 'train':
            label_arr_tr  =np.append(label_arr_tr, label)
            probs_arr_tr = np.append(probs_arr_tr, probs, axis=0)
        elif result[0] == 'val':
            label_arr_val  =np.append(label_arr_val, label)
            probs_arr_val = np.append(probs_arr_val, probs, axis=0)



    ws_tr.append(['Epoch ' + str(epoch)])
    ws_tr.append(label_arr_tr[1:].reshape(-1).tolist())
    ws_tr.append(np.argmax(probs_arr_tr[1:,:], axis=1).reshape(-1).tolist())
    for probs in probs_arr_tr[1:,:].T.tolist():
        ws_tr.append(probs)
    wb_tr.save('./runs/'+logname + '/train.xlsx')
    ws_val.append(['Epoch ' + str(epoch)])
    ws_val.append(label_arr_val[1:].reshape(-1).tolist())
    ws_val.append(np.argmax(probs_arr_val[1:,:], axis=1).reshape(-1).tolist())
    for probs in probs_arr_val[1:,:].T.tolist():
        ws_val.append(probs)
    wb_val.save('./runs/'+logname + '/val.xlsx')
    label_arr_tr = np.zeros((1,1))
    probs_arr_tr = np.zeros((1, nclasses))
    label_arr_val = np.zeros((1,1))
    probs_arr_val = np.zeros((1, nclasses))
    prev_epoch = epoch
    print('Finito')

In [35]:
importlib.reload(ft)

def run_network():
    print(iter_loc)
    '''
    Cretaes the log files and starts the training
    '''
    model_ft, optimizer_ft = network_loader(comment=comment, #'Tested for three rooms'
                                            network=network,
                                            networkName=networkName,
                                            optimizer=optimizer,
                                            iter_loc=iter_loc,
                                            end_to_end=end_to_end,
                                            lr=lr,
                                            momentum=momentum,
                                            weight_decay=weight_decay,
                                            lr_scheduler=lr_scheduler,
                                            lr_decay_epoch=lr_decay_epoch,
                                            pretrained=pretrained,
                                            mse_loss=mse_loss,
                                            nclasses=nclasses)
    
    #model_ft_.fc = nn.Sequential(model_ft.fc, nn.Linear(model_ft.fc.out_features, 1))
    #print(len(optimizer_ft.param_groups))
    
    multi_coeff = make_coeff(nclasses, metric, loss = 'sigmoid', lmbda = coeff_lmbda)
    softmax_matrices = get_soft_matrices(metric, nclasses)
    print('Multi coeff is ' + str(multi_coeff))
    '''Name of the trial'''
    if mse_loss:
        crt='MSE'
    else:
        crt=str(single_loss)+'xsingle + '+str(multi_loss)+'Xmulti'
    logname=network+'_'+'_'+optimizer+'_'+crt+'_'+datetime.now().strftime('%B%d  %H:%M:%S')
    writer = SummaryWriter('../results/tensorboard_runs/'+logname) #For tensorboard
    writeLog(logname)
    writeLog_xlsx()
    
    '''Start trianing'''
    best_model, last_model, result_log = ft.train_model(model_ft,optimizer, lr_scheduler,dset_loaders,
                            dset_sizes,writer,use_gpu=use_gpu,num_epochs=180,batch_size=batch_size,num_log=500,
                            lr_decay_epoch=lr_decay_epoch,init_lr=lr, algo = algo,
                            iter_loc=iter_loc,cross_loss=single_loss,multi_loss=multi_loss,numOut=nclasses,
                            multi_coeff = multi_coeff, single_coeff = multi_coeff, KL = KL, write_log = True,
                            momentum = momentum, weight_decay = weight_decay,
                            logname = '../results/logs.xlsx', cheng_lambda = cheng_lambda, 
                            softmax_matrices = softmax_matrices)
    
    '''Save the models'''
    torch.save(best_model,'../results/saved_models_pytorch/'+logname+'_best')
    torch.save(last_model,'../results/saved_models_pytorch/'+logname+'_last')
    
    print('Writing results')
    #write_epochs(result_log, logname)
    print('Wrote results')
    '''Free up the memory'''
    del model_ft, result_log
    


In [26]:
def get_soft_matrices(metric, n):
    if metric is 'ccr':
        softmax_matrices = [(1, torch.from_numpy(np.eye(n)).type(torch.FloatTensor).to(device))]
    if metric is 'test':
        softmax_matrices = [(1, torch.from_numpy(np.eye(n)).type(torch.FloatTensor).to(device))]
    if metric is 'ccr1':
        mat = np.eye(n)
        for k in range(n-1):
            mat[k, k+1] = 1
            mat[k+1, k] = 1
        softmax_matrices = [(1, torch.from_numpy(mat).type(torch.FloatTensor).to(device))]
    if metric is 'mae' or metric is 'mse':
        softmax_matrices = [(1, torch.from_numpy(np.eye(n)).type(torch.FloatTensor).to(device))]
        for k in range(1, n-1):
            mat = np.eye(n)
            for l in range(n):
                mat[l, max(0,l-k):(k+l+1)] = 1
            if metric is 'mae':
                softmax_matrices.append((2*k + 1, torch.from_numpy(mat).type(torch.FloatTensor).to(device)))
            else:
                softmax_matrices.append((2*k + 1, torch.from_numpy(mat).type(torch.FloatTensor).to(device)))
            
    return softmax_matrices

get_soft_matrices('ccr1', 5)

[(1, tensor([[ 1.,  1.,  0.,  0.,  0.],
          [ 1.,  1.,  1.,  0.,  0.],
          [ 0.,  1.,  1.,  1.,  0.],
          [ 0.,  0.,  1.,  1.,  1.],
          [ 0.,  0.,  0.,  1.,  1.]], device='cuda:0'))]

In [ ]:
a = np.asarray([1,2,3,4])

a[2:8] = 2
a

## 4 - Trials

The loss function we used in the experiments is the following,

$$loss(\mathbf{y},\hat{y})=(1-\lambda )loss_{single}(\mathbf{y},\hat{y})+\lambda loss_{multi}(\mathbf{y},\hat{y})$$
where $\mathbf{y}$ is the ground truth and $\hat{y}$ is the prediction.

Now lets test our function for different values of $\lambda$.

In [ ]:
import copy
model_ft_ = copy.deepcopy(model_ft)
model_ft_.fc = nn.Sequential(model_ft.fc, nn.Softmax(dim=1), nn.Linear(model_ft.fc.out_features, 1, bias = False))

print(model_ft_.fc)

In [ ]:
lr=0.01 #Initial learning rate
optimizer='sgd'
lr_scheduler = ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch= 60 #Number of epoch for learning rate decay
single_loss=0.0
multi_loss=1.0
coeff_lmbda = 1
KL = False

algo = None
metric = 'test'
lr=0.1 #Initial learning rate
run_network()

metric = 'ccr1'
lr=0.1 #Initial learning rate
run_network()
lr=0.01 #Initial learning rate
run_network()

    
    


19
Multi coeff is [[1.         0.97777778 0.95555556 0.93333333 0.91111111 0.88888889
  0.86666667 0.84444444 0.82222222 0.8       ]
 [0.97777778 1.         0.97777778 0.95555556 0.93333333 0.91111111
  0.88888889 0.86666667 0.84444444 0.82222222]
 [0.95555556 0.97777778 1.         0.97777778 0.95555556 0.93333333
  0.91111111 0.88888889 0.86666667 0.84444444]
 [0.93333333 0.95555556 0.97777778 1.         0.97777778 0.95555556
  0.93333333 0.91111111 0.88888889 0.86666667]
 [0.91111111 0.93333333 0.95555556 0.97777778 1.         0.97777778
  0.95555556 0.93333333 0.91111111 0.88888889]
 [0.88888889 0.91111111 0.93333333 0.95555556 0.97777778 1.
  0.97777778 0.95555556 0.93333333 0.91111111]
 [0.86666667 0.88888889 0.91111111 0.93333333 0.95555556 0.97777778
  1.         0.97777778 0.95555556 0.93333333]
 [0.84444444 0.86666667 0.88888889 0.91111111 0.93333333 0.95555556
  0.97777778 1.         0.97777778 0.95555556]
 [0.82222222 0.84444444 0.86666667 0.88888889 0.91111111 0.93333333
  

val Loss: 7.8469 Acc: 0.3131 CIR-1: 0.6251 RMSE 1.8810 MAE 1.3443

Epoch 17/179
----------
500/2188, acc: 0.2988, CIR-1: 0.6278, RMSE: 1.8769, MAE: 1.3493
1000/2188, acc: 0.3009, CIR-1: 0.6257, RMSE: 1.8782, MAE: 1.3510
1500/2188, acc: 0.3029, CIR-1: 0.6246, RMSE: 1.8836, MAE: 1.3532
2000/2188, acc: 0.3033, CIR-1: 0.6238, RMSE: 1.8845, MAE: 1.3542
train Loss: 7.8469 Acc: 0.3033 CIR-1: 0.6236 RMSE 1.8870 MAE 1.3562
val Loss: 7.8458 Acc: 0.3057 CIR-1: 0.6156 RMSE 1.8569 MAE 1.3468

Epoch 18/179
----------
500/2188, acc: 0.3053, CIR-1: 0.6277, RMSE: 1.8610, MAE: 1.3341
1000/2188, acc: 0.3047, CIR-1: 0.6250, RMSE: 1.8730, MAE: 1.3446
1500/2188, acc: 0.3056, CIR-1: 0.6248, RMSE: 1.8833, MAE: 1.3502
2000/2188, acc: 0.3074, CIR-1: 0.6258, RMSE: 1.8797, MAE: 1.3462
train Loss: 7.8440 Acc: 0.3082 CIR-1: 0.6267 RMSE 1.8801 MAE 1.3459
val Loss: 7.8466 Acc: 0.3202 CIR-1: 0.6343 RMSE 1.8913 MAE 1.3368

Epoch 19/179
----------
500/2188, acc: 0.3141, CIR-1: 0.6263, RMSE: 1.8500, MAE: 1.3217
1000/2188

2000/2188, acc: 0.3306, CIR-1: 0.6605, RMSE: 1.7687, MAE: 1.2449
train Loss: 7.8159 Acc: 0.3304 CIR-1: 0.6607 RMSE 1.7711 MAE 1.2469
val Loss: 7.8318 Acc: 0.3369 CIR-1: 0.6558 RMSE 1.8487 MAE 1.2817

Epoch 37/179
----------
500/2188, acc: 0.3294, CIR-1: 0.6617, RMSE: 1.7689, MAE: 1.2416
1000/2188, acc: 0.3308, CIR-1: 0.6632, RMSE: 1.7609, MAE: 1.2374
1500/2188, acc: 0.3312, CIR-1: 0.6630, RMSE: 1.7581, MAE: 1.2364
2000/2188, acc: 0.3318, CIR-1: 0.6632, RMSE: 1.7603, MAE: 1.2369
train Loss: 7.8139 Acc: 0.3317 CIR-1: 0.6635 RMSE 1.7626 MAE 1.2387
val Loss: 7.8321 Acc: 0.3381 CIR-1: 0.6577 RMSE 1.8670 MAE 1.2903

Epoch 38/179
----------
500/2188, acc: 0.3255, CIR-1: 0.6573, RMSE: 1.7650, MAE: 1.2466
1000/2188, acc: 0.3277, CIR-1: 0.6578, RMSE: 1.7733, MAE: 1.2501
1500/2188, acc: 0.3282, CIR-1: 0.6596, RMSE: 1.7711, MAE: 1.2474
2000/2188, acc: 0.3308, CIR-1: 0.6607, RMSE: 1.7667, MAE: 1.2426
train Loss: 7.8135 Acc: 0.3304 CIR-1: 0.6607 RMSE 1.7676 MAE 1.2443
val Loss: 7.8258 Acc: 0.3352 CI

1000/2188, acc: 0.3475, CIR-1: 0.6907, RMSE: 1.6711, MAE: 1.1612
1500/2188, acc: 0.3458, CIR-1: 0.6891, RMSE: 1.6810, MAE: 1.1690
2000/2188, acc: 0.3467, CIR-1: 0.6881, RMSE: 1.6853, MAE: 1.1709
train Loss: 7.7935 Acc: 0.3470 CIR-1: 0.6882 RMSE 1.6876 MAE 1.1723
val Loss: 7.8309 Acc: 0.3371 CIR-1: 0.6615 RMSE 1.8337 MAE 1.2721

Epoch 57/179
----------
500/2188, acc: 0.3412, CIR-1: 0.6869, RMSE: 1.6897, MAE: 1.1743
1000/2188, acc: 0.3435, CIR-1: 0.6867, RMSE: 1.6895, MAE: 1.1745
1500/2188, acc: 0.3449, CIR-1: 0.6853, RMSE: 1.6903, MAE: 1.1755
2000/2188, acc: 0.3446, CIR-1: 0.6863, RMSE: 1.6862, MAE: 1.1738
train Loss: 7.7944 Acc: 0.3456 CIR-1: 0.6859 RMSE 1.6899 MAE 1.1756
val Loss: 7.8284 Acc: 0.3355 CIR-1: 0.6543 RMSE 1.8274 MAE 1.2753

Epoch 58/179
----------
500/2188, acc: 0.3539, CIR-1: 0.6925, RMSE: 1.6710, MAE: 1.1513
1000/2188, acc: 0.3514, CIR-1: 0.6924, RMSE: 1.6738, MAE: 1.1567
1500/2188, acc: 0.3504, CIR-1: 0.6914, RMSE: 1.6787, MAE: 1.1615
2000/2188, acc: 0.3496, CIR-1: 0.6

val Loss: 7.8164 Acc: 0.3489 CIR-1: 0.6713 RMSE 1.7827 MAE 1.2284

Epoch 76/179
----------
500/2188, acc: 0.3729, CIR-1: 0.7256, RMSE: 1.5523, MAE: 1.0572
1000/2188, acc: 0.3743, CIR-1: 0.7278, RMSE: 1.5472, MAE: 1.0555
1500/2188, acc: 0.3706, CIR-1: 0.7242, RMSE: 1.5537, MAE: 1.0643
2000/2188, acc: 0.3702, CIR-1: 0.7249, RMSE: 1.5534, MAE: 1.0640
train Loss: 7.7608 Acc: 0.3697 CIR-1: 0.7242 RMSE 1.5583 MAE 1.0680
val Loss: 7.8164 Acc: 0.3459 CIR-1: 0.6735 RMSE 1.7817 MAE 1.2288

Epoch 77/179
----------
500/2188, acc: 0.3736, CIR-1: 0.7256, RMSE: 1.5520, MAE: 1.0574
1000/2188, acc: 0.3704, CIR-1: 0.7263, RMSE: 1.5511, MAE: 1.0610
1500/2188, acc: 0.3723, CIR-1: 0.7271, RMSE: 1.5518, MAE: 1.0597
2000/2188, acc: 0.3714, CIR-1: 0.7265, RMSE: 1.5495, MAE: 1.0602
train Loss: 7.7597 Acc: 0.3718 CIR-1: 0.7263 RMSE 1.5520 MAE 1.0618
val Loss: 7.8172 Acc: 0.3437 CIR-1: 0.6741 RMSE 1.7798 MAE 1.2299

Epoch 78/179
----------
500/2188, acc: 0.3726, CIR-1: 0.7284, RMSE: 1.5532, MAE: 1.0568
1000/2188

2000/2188, acc: 0.3769, CIR-1: 0.7342, RMSE: 1.5171, MAE: 1.0357
train Loss: 7.7519 Acc: 0.3765 CIR-1: 0.7345 RMSE 1.5183 MAE 1.0371
val Loss: 7.8210 Acc: 0.3459 CIR-1: 0.6728 RMSE 1.7899 MAE 1.2341

Epoch 96/179
----------
500/2188, acc: 0.3763, CIR-1: 0.7411, RMSE: 1.5136, MAE: 1.0279
1000/2188, acc: 0.3768, CIR-1: 0.7385, RMSE: 1.5165, MAE: 1.0319
1500/2188, acc: 0.3774, CIR-1: 0.7387, RMSE: 1.5118, MAE: 1.0303
2000/2188, acc: 0.3777, CIR-1: 0.7371, RMSE: 1.5157, MAE: 1.0327
train Loss: 7.7522 Acc: 0.3775 CIR-1: 0.7371 RMSE 1.5167 MAE 1.0342
val Loss: 7.8219 Acc: 0.3519 CIR-1: 0.6717 RMSE 1.8043 MAE 1.2351

Epoch 97/179
----------
500/2188, acc: 0.3772, CIR-1: 0.7409, RMSE: 1.4866, MAE: 1.0158
1000/2188, acc: 0.3818, CIR-1: 0.7397, RMSE: 1.4974, MAE: 1.0188
1500/2188, acc: 0.3790, CIR-1: 0.7387, RMSE: 1.5052, MAE: 1.0259
2000/2188, acc: 0.3780, CIR-1: 0.7368, RMSE: 1.5125, MAE: 1.0317
train Loss: 7.7511 Acc: 0.3779 CIR-1: 0.7363 RMSE 1.5146 MAE 1.0338
val Loss: 7.8188 Acc: 0.3439 CI

1000/2188, acc: 0.3839, CIR-1: 0.7441, RMSE: 1.4861, MAE: 1.0093
1500/2188, acc: 0.3834, CIR-1: 0.7457, RMSE: 1.4860, MAE: 1.0094
2000/2188, acc: 0.3833, CIR-1: 0.7470, RMSE: 1.4830, MAE: 1.0077
train Loss: 7.7434 Acc: 0.3835 CIR-1: 0.7474 RMSE 1.4825 MAE 1.0078
val Loss: 7.8239 Acc: 0.3536 CIR-1: 0.6670 RMSE 1.8118 MAE 1.2403

Epoch 116/179
----------
500/2188, acc: 0.3931, CIR-1: 0.7549, RMSE: 1.4521, MAE: 0.9788
1000/2188, acc: 0.3870, CIR-1: 0.7498, RMSE: 1.4675, MAE: 0.9960
1500/2188, acc: 0.3846, CIR-1: 0.7491, RMSE: 1.4742, MAE: 1.0012
2000/2188, acc: 0.3844, CIR-1: 0.7483, RMSE: 1.4783, MAE: 1.0035
train Loss: 7.7434 Acc: 0.3839 CIR-1: 0.7476 RMSE 1.4821 MAE 1.0067
val Loss: 7.8268 Acc: 0.3462 CIR-1: 0.6677 RMSE 1.8229 MAE 1.2523

Epoch 117/179
----------
500/2188, acc: 0.3774, CIR-1: 0.7424, RMSE: 1.4986, MAE: 1.0194
1000/2188, acc: 0.3763, CIR-1: 0.7435, RMSE: 1.4920, MAE: 1.0177
1500/2188, acc: 0.3802, CIR-1: 0.7453, RMSE: 1.4895, MAE: 1.0132
2000/2188, acc: 0.3823, CIR-1: 0

val Loss: 7.8247 Acc: 0.3501 CIR-1: 0.6695 RMSE 1.8140 MAE 1.2431

Epoch 135/179
----------
500/2188, acc: 0.3868, CIR-1: 0.7509, RMSE: 1.4560, MAE: 0.9884
1000/2188, acc: 0.3898, CIR-1: 0.7537, RMSE: 1.4572, MAE: 0.9857
1500/2188, acc: 0.3876, CIR-1: 0.7529, RMSE: 1.4590, MAE: 0.9889
2000/2188, acc: 0.3888, CIR-1: 0.7543, RMSE: 1.4567, MAE: 0.9862
train Loss: 7.7366 Acc: 0.3888 CIR-1: 0.7554 RMSE 1.4542 MAE 0.9852
val Loss: 7.8243 Acc: 0.3478 CIR-1: 0.6675 RMSE 1.8007 MAE 1.2405

Epoch 136/179
----------
500/2188, acc: 0.3817, CIR-1: 0.7529, RMSE: 1.4465, MAE: 0.9864
1000/2188, acc: 0.3890, CIR-1: 0.7571, RMSE: 1.4420, MAE: 0.9768
1500/2188, acc: 0.3896, CIR-1: 0.7582, RMSE: 1.4415, MAE: 0.9759
2000/2188, acc: 0.3893, CIR-1: 0.7578, RMSE: 1.4465, MAE: 0.9790
train Loss: 7.7356 Acc: 0.3893 CIR-1: 0.7581 RMSE 1.4480 MAE 0.9801
val Loss: 7.8251 Acc: 0.3470 CIR-1: 0.6671 RMSE 1.8045 MAE 1.2435

Epoch 137/179
----------
500/2188, acc: 0.3928, CIR-1: 0.7566, RMSE: 1.4590, MAE: 0.9795
1000/2

In [15]:
lr=0.1 #Initial learning rate
optimizer='sgd'
lr_scheduler = ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=10 #Number of epoch for learning rate decay
single_loss=0.0
multi_loss=1.0
coeff_lmbda = 1
KL = False

algo = None

metric = 'ccr'

lr=0.05 #Initial learning rate
run_network()
run_network()
run_network()

    


19
Multi coeff is [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Epoch 0/9
----------
LR is set to 0.05
train Loss: 1.9876 Acc: 0.9108 CIR-1: 0.9609 RMSE 0.6152 MAE 0.1566
val Loss: 21.2750 Acc: 0.3513 CIR-1: 0.5818 RMSE 1.9639 MAE 1.4071

Epoch 1/9
----------
train Loss: 0.3123 Acc: 0.9910 CIR-1: 0.9992 RMSE 0.1114 MAE 0.0100
val Loss: 25.8120 Acc: 0.2491 CIR-1: 0.5056 RMSE 1.9458 MAE 1.5558

Epoch 2/9
----------
train Loss: 0.1712 Acc: 0.9950 CIR-1: 0.9996 RMSE 0.0828 MAE 0.0055
val Loss: 23.1010 Acc: 0.3959 CIR-1: 0.6245 RMSE 1.9771 MAE 1.3662

Epoch 3/9
----------
train Loss: 0.1322 Acc: 0.9960 CIR-1: 1.0000 RMSE 0.0634 MAE 0.0040
val Loss: 22.9920 Acc: 0.3680 CIR-1: 0.6097 RMSE 2.1625 MAE 1.4907

Epoch 4/9
----------
train Loss: 0.1225 Acc: 0.9962 CIR-1: 0.9999

Process Process-1146:
Process Process-1148:
Traceback (most recent call last):
  File "/home/mtezcan/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/mtezcan/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/datasets/folder.py", line 103, in __getitem__
    sample = self.transform(sample)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py", lin

  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py", line 175, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py", line 175, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/functional.py", line 204, in resize
    return img.resize((ow, oh), interpolation)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/functional.py", line 204, in resize
    return img.resize((ow, oh), interpolation)
  File "/home/mtezcan/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 1749, in resize
    return self._new(self.im.resize(size, resample, box))
  File "/home/mtezcan/anaconda3/lib/python3.6/si

KeyboardInterrupt: 

In [ ]:
lr=0.001 #Initial learning rate
weight_decay = 0
optimizer='sgd'
lr_scheduler = ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch= 30 #Number of epoch for learning rate decay
single_loss=0.0
multi_loss=1.0
coeff_lmbda = 1
KL = True

algo = 'weighted_softmax_2'


'''metric = 'ccr'
run_network()
metric = 'ccr1'
run_network()'''
metric = 'mae'

lr = 0.001
run_network()


In [ ]:
lr=0.01 #Initial learning rate
optimizer='sgd'
lr_scheduler = ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=20 #Number of epoch for learning rate decay
single_loss=0.0
multi_loss=1.0
coeff_lmbda = 1
KL = True
algo = 'cheng'

cheng_lambda = 0.01
run_network()
cheng_lambda = 0.04
run_network()
cheng_lambda = 0.7
run_network()

In [ ]:
lr=0.03 # Initial learning rate
optimizer='sgd'
lr_scheduler = ft.exp_lr_scheduler # Learning rate scheduler
lr_decay_epoch=20 # Number of epoch for learning rate decay
single_loss=1.0
multi_loss=0.0
coeff_lmbda = 1
KL = True
    
algo = 'binomial'
lr=0.03
run_network()

lr=0.01 # Initial learning rate
run_network()

lr=0.001 # Initial learning rate
run_network()


In [ ]:
for k in range(5):
    metric = 'ccr'
    KL = True
    single_loss=1.
    multi_loss=0.
    run_network()

In [ ]:
KL = True

metric = 'ccr'
single_loss= 1.0
multi_loss = 0.0
run_network()

metric = 'ccr1'
run_network()

metric = 'mae'
run_network()

metric = 'mse'
run_network()

metric = 'ccr1'
for lmbda in [.2, .4, .6, .8, .9]:
    single_loss= round(1.-lmbda, 1)
    multi_loss = lmbda
    print('Single loss = '+str(single_loss)+', Multi loss = '+str(multi_loss))
    run_network()
    
metric = 'mae'
for lmbda in [.2, .4, .6, .8, .9]:
    single_loss= round(1.-lmbda, 1)
    multi_loss = lmbda
    print('Single loss = '+str(single_loss)+', Multi loss = '+str(multi_loss))
    run_network()

    

In [ ]:
slr=0.01 #Initial learning rate
momentum=0.9
weight_decay=0.0005
lr_scheduler=ft.exp_lr_scheduler #Learning rate scheduler
lr_decay_epoch=10 #Number of epoch for learning rate decay
single_loss= 1.0
multi_loss = 0.0
metric = 'ccr'
run_network()
metric = 'ccr1'
run_network()
metric = 'mae'
run_network()
metric = 'mse'
run_network()

In [ ]:
single_loss=0.
multi_loss=1.

for k in range(len(dset_loaders_arr)):
    dset_loaders = dset_loaders_arr[k]
    run_network()
    
single_loss=.1
multi_loss=.9

for k in range(len(dset_loaders_arr)):
    dset_loaders = dset_loaders_arr[k]
    run_network()

In [ ]:
multi_coeff = (1/5) * [1, 2, 3, 4, 5, 4, 3, 2, 1]
run_network()

In [ ]:
multi_coeff = np.asarray([1])
print(multi_coeff)
run_network()

single_loss=1.
multi_loss=0.
run_network()

single_loss = 0.1
multi_loss = 0.9
multi_coeff = [1, 1, 1]
run_network()

In [ ]:
'''multi_coeff = (1/5) * np.asarray([1, 2, 3, 4, 5, 4, 3, 2, 1])
print(multi_coeff)
run_network()'''

multi_coeff = (1/25) * np.asarray([9, 16, 21, 24, 25, 24, 21, 16, 9])
print(multi_coeff)
run_network()

multi_coeff = np.asarray([1])
print(multi_coeff)
run_network()

single_loss=1.
multi_loss=0.
run_network()

In [ ]:
single_loss = 0.1
multi_loss = 0.9
multi_coeff = (1/5) * np.asarray([1, 2, 3, 4, 5, 4, 3, 2, 1])
print(multi_coeff)
run_network()

In [ ]:
single_loss = 0.0
multi_loss = 1.0
multi_coeff =  np.asarray([.9, 1, .9])
print(multi_coeff)
run_network()

In [ ]:
multi_coeff = (1/25) * np.asarray([9, 16, 21, 24, 25, 24, 21, 16, 9])
extend = int((len(multi_coeff) - 1) / 2)
label_multi = np.zeros(numOut + 2 * extend)
label_multi[label:label + 2 * extend + 1] = multi_coeff

if extend is not 0:
    label_multi = label_multi[extend:-extend]

print(label_multi)

In [ ]:
mse_loss=False
for lmbda in [1., 1., 1., 1., 1.]:
    single_loss= round(1.-lmbda, 1)
    multi_loss = lmbda
    print('Single loss = '+str(single_loss)+', Multi loss = '+str(multi_loss))
    run_network()